In [15]:
import os
import sys
import numpy as np
from os import listdir
from os.path import isfile, isdir, join
import wave
import struct
from tqdm import tqdm
import soundfile as sf
import wavio

## How to get file extension
>>> import os
>>> filename, file_extension = os.path.splitext('/path/to/somefile.ext')
>>> filename
'/path/to/somefile'
>>> file_extension
'.ext'

In [2]:
min_value = np.iinfo('int16').min
max_value = np.iinfo('int16').max

## get specific type of files


You can use glob:

import glob, os
os.chdir("/mydir")
for file in glob.glob("*.txt"):
    print(file)

or simply os.listdir:

import os
for file in os.listdir("/mydir"):
    if file.endswith(".txt"):
        print(os.path.join("/mydir", file))

or if you want to traverse directory, use os.walk:

import os
for root, dirs, files in os.walk("/mydir"):
    for file in files:
        if file.endswith(".txt"):
             print(os.path.join(root, file))



In [3]:
def testFileExt(trueExt,filefullpath):
    filename, file_extension = os.path.splitext('/path/to/somefile.ext')
    if file_extension == trueExt:
        return True
    else:
        return False

In [4]:
def getFilesInFloder(folderPath):
    onlyfiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f))]
    return onlyfiles

def getDirsInFolder(baseDirPath):
    onlySubDirs = [d for d in listdir(baseDirPath) if isdir(join(baseDirPath, d))]
    return onlySubDirs

In [7]:
file_dir = "../../speechData/noise_for_test_20200319/"#"D:/MyGitPrjs/2018_Acustic_Resource_Classes/noise_dat/"
files = getFilesInFloder(file_dir)

In [8]:
def Dat2WAV(filesPath=None):
    sampleRate=16000
    for file in tqdm(files):
        #read dat file
        if file.endswith(".dat"):
            tmp_list = []
            with open(file_dir+file,"r") as rf:
                tmp_list = rf.readlines()
                
            print("file:{} length is {}".format(file,len(tmp_list)))
            with wave.open("../../speechData/noise_for_test_20200319/converted/{}.wav".format(file),"w")as obj:
                obj.setnchannels(1) # mono
                obj.setsampwidth(2)
                obj.setframerate(sampleRate)
                tmp_list = tmp_list[1:]
                for value in tmp_list:
                    value = value.replace("\n","")
                    value = int(float(value))
                    data = struct.pack('<h', value)
                    obj.writeframesraw(data)
    print("processing completed.")
        

In [10]:
Dat2WAV()

 20%|██        | 1/5 [00:00<00:00,  5.48it/s]

file:上升.dat length is 160001
file:驢.dat length is 160001


 60%|██████    | 3/5 [00:00<00:00,  5.68it/s]

file:開場.dat length is 160001
file:波浪.dat length is 160001


100%|██████████| 5/5 [00:00<00:00,  5.77it/s]

file:象.dat length is 160001
processing completed.


In [20]:
ffmpeg_mp3_to_wav_cmd = "ffmpeg -i {mp3} -vn -acodec pcm_s16le -ac 1 -ar {sr} -f wav {wav}"
mpg123_mp3_to_wav_cmd = "mpg123 -w {wav} {mp3}"

In [21]:
import subprocess
def run_win_cmd(cmd):
    result = []
    process = subprocess.Popen(cmd,
                               shell=True,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    for line in process.stdout:
        result.append(line)
    errcode = process.returncode
    for line in result:
        print(line)
    if errcode is not None:
        raise Exception('cmd %s failed, see above for details', cmd)

In [28]:
noise_mp3_files_path = "D:/MyGitPrjs/2018_Acustic_Resource_Classes/noise_mp3/"
speech_mp3_files_path = "D:/MyGitPrjs/2018_Acustic_Resource_Classes/speech_only_mp3/"
mp3_file_list = getFilesInFloder(noise_mp3_files_path)
speech_mp3_file_list = getFilesInFloder(speech_mp3_files_path)
def convert_mp3_to_wav():
    for f in tqdm(speech_mp3_file_list):
#         print("processing {}.....".format(f))
        command = ffmpeg_mp3_to_wav_cmd.format(mp3=speech_mp3_files_path+f, sr=16000, wav="D:/MyGitPrjs/2018_Acustic_Resource_Classes/speech_only_wav/two_channel_wav/{}.wav".format(f))
        wcommand = 'cmd /c "{}"'.format(command)        
#         print(wcommand)
        os.system(wcommand)


In [29]:
convert_mp3_to_wav()

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:05<00:00,  4.09it/s]


In [23]:
def convert_two_channel_wav_to_mat(src_dir,filename,mat_save_dir, chnum=2):
    mat_len = 160000
    #read wav file
    audio_array, sr  = sf.read(src_dir+filename)#librosa.load(filename)
    if chnum==2:
        audio_array = audio_array[:,0]
    ret_clip = (audio_array[0:mat_len]*max_value).astype(dtype='double')
    mat_dict = {"s":ret_clip}
    spio.savemat(mat_save_dir+filename+".mat",mat_dict,oned_as='column')
    print("converting {} to mat file completed.".format(filename))

In [17]:
def convert_wav_to_array(wavfile:str=None):
    # audio_array, sr  = sf.read(wavfile);
    audio_array = wavio.read(wavfile).data.T[0];
    wav_len = len(audio_array);
    print(f"{wavfile} length:{wav_len}");
    if  wav_len < 20150:
        audio_array = np.pad(audio_array,(0,20150 - wav_len),'constant');
    return audio_array;

In [18]:
to_convert_wav = "../uec_aed/dataset/raw/wanted_beep_files/beep_clear_clip_01_22K.wav";
wav_array = convert_wav_to_array(wavfile=to_convert_wav);
print(len(wav_array))

../uec_aed/dataset/raw/wanted_beep_files/beep_clear_clip_01_22K.wav length:7199
20150


In [22]:
col_num = 20;
r_brace ='{'
l_brace = '}'
comma_sym = ',';
full_angle_brackets = '[]';
eq_sym = '='
semicolon_sym = ';'
c_const_char_arr_dec = 'const signed char';
c_array_name = 'wav_array'
##output wav array to c char array
print(f"{c_const_char_arr_dec} {c_array_name}{full_angle_brackets}={r_brace}");
line_count = 0;
ele_count = 0;
str_buffer = ""
len_of_wav = len(wav_array);
while ele_count < len_of_wav:
    ary_start = line_count * col_num
    ary_end = (line_count+1) * col_num
    for e in wav_array[ary_start:ary_end]:
        str_buffer = str_buffer+str(e)+", "
        ele_count += 1;
    line_count += 1
    print(str_buffer);
    str_buffer = "";
print("};")

const signed char wav_array[]={
-146, -90, -70, 22, 183, 164, 116, 93, 239, 182, 106, 21, 67, 196, 255, 260, 308, 319, 433, 361, 
291, 296, 70, 161, 226, 90, 45, 54, -68, -32, -57, -7, 188, 297, 245, 157, 226, 306, 283, 349, 
328, 165, 142, 87, 146, 134, 212, 368, 314, 295, 209, 155, 175, 106, 171, 77, -28, -34, -27, 9, 
-128, 154, 213, 99, 88, 0, -16, 5, 26, 23, 42, -14, -53, -162, -215, -195, -128, -13, 55, 43, 
134, 253, 107, 152, 211, 274, 273, 78, -96, -56, 26, -96, -127, -136, 9, 25, -15, 137, -12, -80, 
-3, -23, -109, -80, -66, -96, -148, 32, 78, 143, 254, 178, 286, 137, 36, -49, -88, -74, -175, -252, 
-150, -27, -75, -55, -15, 29, 139, 9, 47, 67, 20, 86, -45, -111, -308, -235, -143, -57, -76, 9, 
226, 81, -155, -213, -134, -66, -60, -120, -40, -48, 70, 16, 69, 15, -57, 231, 62, -87, -131, -37, 
-7, -40, -78, -64, -104, -84, -27, -117, -97, -59, -261, -248, -52, -68, -134, -66, 172, 177, 179, 240, 
136, 223, 156, 54, 114, 81, 80, 84, 119, 343, 344, 208, 357, 326, 270, 202, 210, 